In [1]:
%pylab inline
import pickle
import glob
import os
import pandas as pd
import numpy as np
import sys

sys.path.insert(0, ".")

import seaborn as sns

sns.set()
sns.set_style("whitegrid")
sns.set_context("paper")


def pd_full(rows=100):
    pd.set_option("display.max_rows", rows)
    pd.set_option("display.max_columns", None)
    pd.set_option("display.width", 2000)
    pd.set_option("display.float_format", "{:20,.3f}".format)
    pd.set_option("display.max_colwidth", None)


pd_full()

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Load relevant experiment script
import scripts.run_base_configs_exp as exp
from filternet.training.evalmodel import *

In [3]:
from filternet.datasets.opportunity import get_x_y_contig
from filternet.datasets import sliding_window_x_y
from torch.utils.data import DataLoader, TensorDataset

# REPLACE WITH NEW MODEL
model_dir = './scripts/saved_models/base_configs_7_multi_scale_cnn_lstm_1'
em = load_eval_model_from_dir(model_dir)
# em.run_test_set()
# em.calc_metrics()
# em.calc_ward_metrics()

display(em.classification_report_df)
em.f1, em.f1_mean, em.infer_samples_per_s, em.n_samples_in, em.n_samples_out


C:\Users\afsamani\Anaconda3\envs\ML\lib\site-packages\torch\optim\lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


,precision,recall,f1-score,support,event_f1
,0.963,0.959,0.961,12337,0.922
Open Door 1,0.621,0.857,0.720,84,0.667
Open Door 2,0.930,0.833,0.879,144,0.933
Close Door 1,0.728,0.670,0.698,88,0.714
Close Door 2,0.827,0.992,0.902,125,0.889
Open Fridge,0.872,0.660,0.751,341,0.931
Close Fridge,0.787,0.907,0.843,236,0.982
Open Dishwasher,0.659,0.738,0.696,149,0.727
Close Dishwasher,0.671,0.831,0.742,118,0.737
Open Drawer 1,0.683,0.695,0.689,59,0.632


(0.9260763034888204, 0.7455457850458616, 58715.49246745205, 57856, 14816)

In [4]:
Xc, ycs, data_spec = get_x_y_contig('test')
dls = {}
win_lens = [
    32, 
    64, 
    128, 
    256, 
    512, 
    1024, 
    1024*2, 
    1024*4, 
    1024* 8, 
     1024*16
]

new_len = int(floor(len(Xc) / np.max(win_lens)) * np.max(win_lens))
print(f'Cropping data from {len(Xc)} to {new_len} for consistency between win_lens')
Xc = Xc[:new_len]
ycs = [yc[:new_len] for yc in ycs]

for win_len in win_lens:
    print(win_len)
    # Properly window data for this model
    X, ys = sliding_window_x_y(
        Xc,
        ycs,
        win_len=win_len,
        step=int(win_len/2),
        shuffle=False,  
    )

    dls[win_len] = DataLoader(
        TensorDataset(torch.Tensor(X), *[torch.Tensor(y).long() for y in ys]),
        batch_size=em.eval_batch_size,
        shuffle=False,
    )

e:\OneDrive - Aalborg Universitet\WrokResults\FilterNet\.\filternet\datasets\opportunity.py:84: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  (


Opportunity data not cached. Creating cache now...
Test10!!!!!!
dataset/S1-ADL4.dat
dataset/S1-Drill.dat
dataset/S1-ADL5.dat
dataset/S1-ADL1.dat
dataset/S1-ADL2.dat
dataset/S1-ADL3.dat
dataset/S2-ADL2.dat
dataset/S3-ADL2.dat
dataset/S3-ADL1.dat
dataset/S2-ADL1.dat
dataset/S3-Drill.dat
dataset/S2-Drill.dat
dataset/S2-ADL3.dat
dataset/S3-ADL3.dat
dataset/S2-ADL4.dat
dataset/S2-ADL5.dat
dataset/S3-ADL4.dat
dataset/S3-ADL5.dat
Caching done.
Loading cached data.
Loaded.
Cropping data from 118750 to 114688 for consistency between win_lens
32
64
128
256
512
1024
2048
4096
8192
16384


In [ ]:
results = []

In [ ]:

for base_config in ['base_cnn', 'multi_scale_cnn','base_lstm',
       'cnn_lstm',  
                    'multi_scale_cnn_lstm']:
    for i_run in range(2,7):
        model_dir = f'saved_models/base_configs_7_{base_config}_{i_run}'
        print(model_dir)
        em = load_eval_model_from_dir(model_dir)
        
        for win_len in win_lens:
            print(win_len)

            em.cuda = True
            em.device = 'cuda'
            em.model.cuda()
            em.run_test_set(dls[win_len])
            cuda_infer_rate = em.infer_samples_per_s

            em.cuda = False
            em.device = 'cpu'
            em.model.cpu()
            em.run_test_set(dls[win_len])
            cpu_infer_rate = em.infer_samples_per_s

            # Run on this data subset
            em.run_test_set(dls[win_len])
            # Calc new metrics for *just* this subset
            em.calc_metrics()
            em.calc_ward_metrics()

        #     display(em.classification_report_df)
            print(em.f1, em.f1_mean, em.event_f1, int(em.infer_samples_per_s/1000), em.n_samples_in, em.n_samples_out)
            results.append(dict(
                win_len=win_len,
                i_run=i_run,
                model_dir=model_dir,
                base_config=em.trainer.base_config,
                f1=em.f1, 
             f1_mean=em.f1_mean,
             event_f1=em.event_f1,
#              fer_samples_per_s=em.infer_samples_per_s,
             cuda_infer_rate = cuda_infer_rate,
             cpu_infer_rate  = cpu_infer_rate
            ))


In [ ]:
'a' # 4204

In [ ]:
df_results = pd.DataFrame(results)
df_results

In [ ]:
df_results.groupby(['base_config', 'win_len']).mean().sort_index().drop('i_run', axis=1)

In [ ]:
# CHECKPOINT
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
fn =  f"win_len_timings_df_{timestr}.pkl"
df_results.to_pickle(fn)
fn

In [ ]:
raise Exception() # So we don't automatically run through this
# must change filename below to match above.

In [ ]:
fn = 'win_len_timings_df_20200209-153419.pkl'
df = pd.read_pickle(fn)
df.cuda_infer_rate /= 1000
df.cpu_infer_rate /= 1000

In [ ]:
df.base_config.unique()

In [ ]:
df.groupby(['base_config', 'win_len']).i_run.count()

In [ ]:
df.groupby(['base_config', 'win_len']).i_run.max()

In [ ]:
df2 = df.groupby(['base_config', 'win_len']).mean().sort_index().drop('i_run', axis=1)
df2

In [ ]:
df3 = df2.loc['base_cnn']

In [ ]:
configs = [
    ['base_lstm'],
    ['base_cnn'],
    ['cnn_lstm'],
     ['multi_scale_cnn'], 
    ['multi_scale_cnn_lstm'],
]
names = [
    'b-LSTM',
    'p-CNN',
    'p-C/L',
    'ms-CNN',
    'ms-C/L'
]
def set_legend(legend_names, legend_title=None, ax=None):
    if ax is None:
        ax = gca()
    ax.get_legend().set_title(legend_title)
    # replace labels
    for t, l in zip(ax.get_legend().texts, legend_names): 
        t.set_text(l)

scale = .8
kwargs = dict( kind='point', dodge=True,
              ci="sd", capsize=.2, linestyles=['-', '--', '-.'], 
              markers = ['o', '.', '^'], scale=scale)

f, axes = subplots(2,len(configs), figsize=[1.6 * len(configs),4.25], dpi=300, sharex=True)
for i, (base_config,) in enumerate(configs):
    df5 = df2.loc[base_config].iloc[:-1,:]

    sca(axes[0][i])
    gca().set_xscale('log')
    
    plot(df5.index, df5.f1, 'o-', color=sns.palettes.color_palette()[0], label='$F_{1w}$')
    plot(df5.index, df5.event_f1, '.--', color=sns.palettes.color_palette()[1], label='$F_{1e}$')
    plot(df5.index, df5.f1_mean, '^-.', color=sns.palettes.color_palette()[2], label='$F_{1m}$')

    
#     ff = sns.pointplot(data=df3, x="win_len", y="value", hue='Metric',
#                        hue_order=["f1", "event_f1", "f1_mean"],
#                        **kwargs)

#     set_legend(['F1 (weighted)', 'F1 (mean)', 'Event F1'])

    title(names[i])
    ylim([.4, 1])
    xlabel(None)
    if i == len(configs)-1:
        ylabel('F1 metric value')
        legend(loc='lower right')
    
    gca().label_outer()
    
#     df3 =  pd.melt(df.query('base_config == @base_config'), id_vars=['base_config', 'win_len'], value_vars=['cuda_infer_rate', 'cpu_infer_rate'],
#                   var_name='Metric')
    sca(axes[1][i])
#     ff = sns.pointplot(data=df3, x="win_len", y="value", hue='Metric',**kwargs)
#     set_legend(['F1 (weighted)', 'Event F1'], #'Metric'
#         )
    plot(df5.index, df5.cuda_infer_rate, 'o-', color=sns.palettes.color_palette()[0], label='GPU')
    plot(df5.index, df5.cpu_infer_rate, '^--', color=sns.palettes.color_palette()[1], label='CPU')


    ylim([0, 1500])
#     xlim([0, None])

    xticks(df5.index[::2], [str(w) for w in df5.index[::2]], rotation=30)
    xlabel('Window length')
    if i == len(configs)-1:
        ylabel('Infer Rate [ksamp/s]')
        legend(loc='upper right')
        
    gca().label_outer()
    
    
tight_layout(w_pad=0.5)#w_pad=3)
plt.savefig('win_len_effects.png')

In [ ]:
df5